# Bag of Tricks Experiment

Analyze the effects of our different "tricks".

1. Sample matches off mask
2. Scale by hard negatives
3. L2 pixel loss on matches

We will compare standard network, networks missing one trick only, and a network without any tricks (i.e same as Tanner Schmidt)

In [ ]:
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import logging

# utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
logging.basicConfig(level=logging.INFO)


In [ ]:
dataset_config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', "caterpillar_baymax_starbot_all_front_single_only.yaml")

train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'training.yaml')



logging_dir = "code/data_volume/pdc/trained_models/trick_analysis"
num_iterations = 3500
num_image_pairs = 100
debug = False

TRAIN = True
EVALUATE = True


# num_image_pairs = 10
# num_iterations = 10

d = 3

network_dict = dict()


## Standard

In [ ]:
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)
dataset = SpartanDataset(config=dataset_config)


train_config = utils.getDictFromYamlFilename(train_config_file)

name = "standard_%d" %(d)
print "training %s" %(name)
train_config = utils.getDictFromYamlFilename(train_config_file)
train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = name
train._config["training"]["num_iterations"] = num_iterations
train._config["dense_correspondence_network"]["descriptor_dimension"] = d


if TRAIN:
    train.run()
print "finished training descriptor of dimension %d" %(d)

 # now do evaluation
print "running evaluation on network %s" %(name)
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
network_dict[name] = model_folder
if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)
print "finished running evaluation on network %s" %(name)

## With L2 on masked non_matches

In [ ]:
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)
dataset = SpartanDataset(config=dataset_config)

train_config = utils.getDictFromYamlFilename(train_config_file)
name = "l2_masked_%d" %(d)
print "training %s" %(name)
train_config = utils.getDictFromYamlFilename(train_config_file)
train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = name
train._config["training"]["num_iterations"] = num_iterations

train._config["dense_correspondence_network"]["descriptor_dimension"] = d

train._config["loss_function"]["use_l2_pixel_loss_on_masked_non_matches"] = True



if TRAIN:
    train.run()
print "finished training descriptor of dimension %d" %(d)

 # now do evaluation
print "running evaluation on network %s" %(name)
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
network_dict[name] = model_folder
if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)
print "finished running evaluation on network %s" %(name)

## Dont scale by hard negatives

In [ ]:
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

dataset = SpartanDataset(config=dataset_config)
train_config = utils.getDictFromYamlFilename(train_config_file)

name = "dont_scale_hard_negatives_%d" %(d)
print "training %s" %(name)
train_config = utils.getDictFromYamlFilename(train_config_file)
train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = name
train._config["training"]["num_iterations"] = num_iterations
train._config["dense_correspondence_network"]["descriptor_dimension"] = d

train._config["loss_function"]["scale_by_hard_negatives"] = False


if TRAIN:
    train.run()
print "finished training descriptor of dimension %d" %(d)

 # now do evaluation
print "running evaluation on network %s" %(name)
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
network_dict[name] = model_folder
if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)
print "finished running evaluation on network %s" %(name)

## Dont sample off mask

In [ ]:
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

dataset = SpartanDataset(config=dataset_config)
train_config = utils.getDictFromYamlFilename(train_config_file)

name = "dont_sample_from_mask_%d" %(d)
print "training %s" %(name)
train_config = utils.getDictFromYamlFilename(train_config_file)
train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = name
train._config["training"]["num_iterations"] = num_iterations
train._config["dense_correspondence_network"]["descriptor_dimension"] = d

train._config["training"]["sample_matches_only_off_mask"] = False
train._config["training"]["use_image_b_mask_inv"] = False
train._config["training"]["fraction_masked_non_matches"] = 0.01
train._config["training"]["fraction_background_non_matches"] = 0.99


if TRAIN:
    train.run()
print "finished training descriptor of dimension %d" %(d)

 # now do evaluation
print "running evaluation on network %s" %(name)
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
network_dict[name] = model_folder
if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)
print "finished running evaluation on network %s" %(name)

## No tricks

In [ ]:
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

dataset = SpartanDataset(config=dataset_config)
train_config = utils.getDictFromYamlFilename(train_config_file)

name = "no_tricks_%d" %(d)
print "training %s" %(name)
train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train_config = utils.getDictFromYamlFilename(train_config_file)
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = name
train._config["training"]["num_iterations"] = num_iterations
train._config["dense_correspondence_network"]["descriptor_dimension"] = d

train._config["loss_function"]["scale_by_hard_negatives"] = False
train._config["loss_function"]["use_l2_pixel_loss_on_masked_non_matches"] = False
train._config["training"]["sample_matches_only_off_mask"] = False

train._config["training"]["use_image_b_mask_inv"] = False
train._config["training"]["fraction_masked_non_matches"] = 0.01
train._config["training"]["fraction_background_non_matches"] = 0.99



if TRAIN:
    train.run()
print "finished training descriptor of dimension %d" %(d)

 # now do evaluation
print "running evaluation on network %s" %(name)
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
network_dict[name] = model_folder
if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)
print "finished running evaluation on network %s" %(name)

## L2 and dont scale hard negatives

In [ ]:
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

dataset = SpartanDataset(config=dataset_config)
train_config = utils.getDictFromYamlFilename(train_config_file)

name = "l2_dont_scale_hard_negatives_run_2_%d" %(d)
print "training %s" %(name)
train_config = utils.getDictFromYamlFilename(train_config_file)
train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
train._config["training"]["logging_dir"] = logging_dir
train._config["training"]["logging_dir_name"] = name
train._config["training"]["num_iterations"] = num_iterations
train._config["dense_correspondence_network"]["descriptor_dimension"] = d

train._config["loss_function"]["scale_by_hard_negatives"] = False
train._config["loss_function"]["use_l2_pixel_loss_on_masked_non_matches"] = True


if TRAIN:
    train.run()
print "finished training descriptor of dimension %d" %(d)

 # now do evaluation
print "running evaluation on network %s" %(name)
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_to_absolute_path(model_folder)
network_dict[name] = model_folder
if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)
print "finished running evaluation on network %s" %(name)